# 무신사 selenium 페이지 순회
> selenium을 이용하여 무신사 사이트를 페이지 이동하면서 스크래핑하는 실습을 정리하였습니다

- toc: true
- badges: true
- comments: true
- categories: [crawling]
- image:

In [84]:
import numpy as np
import pandas as pd
import requests
import time
import tqdm
import warnings
import re

import random

from bs4 import BeautifulSoup
from selenium import webdriver
from tqdm import notebook

In [124]:
warnings.filterwarnings(action='ignore')

## 페이지 이동하면서 크롤링

<img src = "{{site.baseurl}}/images/crawling/상품리스트.png" width = "60%" height = "60%">

<img src = "{{site.baseurl}}/images/crawling/html 구조.png" width = "60%" height = "60%">

In [89]:
#
# 무신사 접속
# 카테고리 url 접속

url = 'https://www.musinsa.com/category/001001'
driver = webdriver.Chrome('/Users/yoohajun/Library/Mobile Documents/com~apple~CloudDocs/Hajun/scrapy/musinsa/chromedriver')
driver.implicitly_wait(3)  # 웹 페이지 로딩 최대 5초 대기
driver.get(url)

### url을 순회할 상품 코드들을 얻어오자

- 카테고리 페이지(상의-전체)에는 상품들이 나열되어 있다
- 해당 상품들은 goods code를 가지고 있는데
- 클릭해서 상품 detail로 가보면 goods code를 가지고 url이 반복되고 있는 것을 알 수 있다.
- https://www.musinsa.com/app/goods/{goods_code}
- 카테고리 페이지에서 data-no 태그를 통해서 우리는 페이지 내의 상품들의 `goods code`를 가지고 와 리스트 형식으로 저장할 것 이다.



In [90]:
#
# 상품 - 페이지 번호
# 카테고리 페이지 -> https://www.musinsa.com/category/001001
# css #searchList > li
# # searchList > li:nth-child(2)
# 리스트로 가져오기 
# 1848166, 1921901 등등
# 인덱스 오류를 방지하기 위해 반복문을 순회할 때 예외처리 코드를 넣어준다. 

code_list = list()

for i in range(1,91):
    css_selector = f'#searchList > li:nth-child({i})'
    try : 
        data_list = driver.find_element_by_css_selector(css_selector)
        # data-no 태그를 가져온다 
        data_no = data_list.get_attribute('data-no') 
        code_list.append(data_no)
    except :
        print('data_no out of index')

code_list

['1848166',
 '1921901',
 '996177',
 '1841764',
 '2442409',
 '996178',
 '1884943',
 '2471760',
 '2034137',
 '1420730',
 '903340',
 '2035287',
 '2391261',
 '1911516',
 '2479911',
 '2453556',
 '1388775',
 '2402005',
 '1939099',
 '2409894',
 '1503352',
 '1860490',
 '751030',
 '2368712',
 '996497',
 '2086653',
 '1779737',
 '345889',
 '1899755',
 '2426596',
 '2500583',
 '2329824',
 '987149',
 '1382658',
 '1886950',
 '792918',
 '1955452',
 '2376229',
 '1417691',
 '2456003',
 '996498',
 '2398381',
 '2462040',
 '1388776',
 '2015684',
 '1793420',
 '1882396',
 '1820701',
 '1236364',
 '2457846',
 '2396221',
 '2082054',
 '1782647',
 '324416',
 '1417715',
 '538899',
 '2401885',
 '751029',
 '746280',
 '1944612',
 '2453552',
 '1893766',
 '747950',
 '2406942',
 '2403601',
 '1816729',
 '746276',
 '2274644',
 '1846930',
 '1375522',
 '1827196',
 '1417692',
 '2409397',
 '1728662',
 '2540678',
 '2377269',
 '2487719',
 '1685307',
 '1845485',
 '1827193',
 '1943792',
 '1403092',
 '1454812',
 '748056',
 '141057

### detail 상품 페이지 스크래핑

In [131]:
#

# 우선 base url을 만들어서 해당 url로 이동을 해준다
# https://www.musinsa.com/app/goods/{code_list}
base_url = "https://www.musinsa.com/app/goods/"

driver = webdriver.Chrome('/Users/yoohajun/Library/Mobile Documents/com~apple~CloudDocs/Hajun/scrapy/musinsa/chromedriver')

driver.implicitly_wait(5)  # 웹 페이지 로딩 최대 5초 대기
driver.get(base_url)



In [122]:
# 리스트 - dict 형식으로 저장하자
item_list = list()

- https://www.musinsa.com/app/goods/{상품 코드}
- 위의 상품 코드를 미리 만들어놓은 리스트 원소를 대입하면서 반복문을 돌리자
- 상품마다 동일한 코드가 적용되기 어렵다
    - 예외가 많기 때문에 예외 처리가 필수적으로 동반되어야한다.
    - 페이지마다 특정 요소들이 동일한 위치에 있지 않고 약간 변경되거나
    - 어떤 요소들은 문자열 패턴이 다양해서 예외처리를 하지 않고 그대로 가져와야하는 경우도 발생한다
        - 2022 S/S / 남 여 or 남 여 or ALL ALL / 남
    - 예외처리는 `try - except` 구문을 이용해서 진행

<img src = "{{site.baseurl}}/images/crawling/무신사 크롤링.png" width = "60%" height = "60%">

In [143]:
#

# 세부 상품 크롤링을 진행하자

## 상품 코드 번호를 순회하며 url에 대입 후 스크래핑

for idx in tqdm(code_list) :
    
    # 아이템 객체 생성 
    item = dict()
    
    # url을 변경해서 get해온다
    item_url = base_url+idx
    driver.get(item_url)
    
    # url 변경 후 2초간 대기
    time.sleep(2)
    
    
    # 상세 페이지 옷 사진 가져오기

    # img css selector을 가져온 후 
    image = driver.find_element_by_css_selector('#detail_bigimg > div.product-img > img')
    
    # src 태그를 가져온다 
    src = image.get_attribute('src') 
    
    item['src'] = src
    print(src)
    
    # 상품 이름 
    product_name = driver.find_element_by_css_selector('#page_product_detail > div.right_area.page_detail_product > div.right_contents.section_product_summary > span > em').text
    print(product_name)
    
    # #product_order_info > div.explan_product.product_info_section > ul > li:nth-child(1) > p.product_article_contents > strong
    # 브랜드 이름과 품번 가져오기

    product_brand = driver.find_element_by_css_selector('#product_order_info > div.explan_product.product_info_section > ul > li:nth-child(1) > p.product_article_contents > strong').text
    brand_id = product_brand.split('/') 
    brand_id = [elem.strip() for elem in brand_id]
    brand_name = brand_id[0]
    product_id = brand_id[1]
    print(brand_name)
    print(product_id)
    item['brand_name'] = brand_name
    item['product_id'] = product_id
    
    
    # 시즌 정보와 성별 가져오기
    season_gender = driver.find_element_by_css_selector('#product_order_info > div.explan_product.product_info_section > ul > li:nth-child(2) > p.product_article_contents').text

    item['season_gender'] = season_gender
    print(season_gender)  

    # 상품 가격 가져오기

    price = driver.find_element_by_css_selector('#goods_price').text
    price = re.sub('[-=.,#/?:$}원]', '', price)
    print(price)
    item['price'] = price
    
    # 해시태그 가져오기
    
    ## 해시태그가 없는 상품도 존재하기 때문에 예외처리가 필요하다.

    ## #product_order_info > div.explan_product.product_info_section > ul > li.article-tag-list.list > p

    try : 

        hashtag = driver.find_element_by_css_selector('#product_order_info > div.explan_product.product_info_section > ul > li.article-tag-list.list > p').text

        hashtag = hashtag.split('\n')

        # 정규표현식을 통해 #(특수기호) 제거
        hashtag = [re.sub('[-=.#/?:$}]', '', elem) for elem in hashtag]
        print(hashtag)
        item['hashtag'] = hashtag

    except :
        hashtag = None
        print(hashtag)
        item['hashtag'] = hashtag
    
    # 좋야요 개수
    # # product-top-like > p.product_article_contents.goods_like_{상품 코드} > span
    
    temp_selector = f'#product-top-like > p.product_article_contents.goods_like_{idx} > span'

    like = driver.find_element_by_css_selector(temp_selector).text
    like = re.sub('[^0-9]', '', like)
    print(like)
    item['like'] = like
    
    # 평점
    # 평점은 list child의 위치가 다르게 나올 수 있기 때문에 예외처리를 2가지 케이스로 나눠서 해보았다.
    
    try :
        rate = driver.find_element_by_css_selector('#product_order_info > div.explan_product.product_info_section > ul > li:nth-child(6) > p.product_article_contents > a > span.prd-score__rating').text
        print(rate)
        item['rate'] = rate
  
    except : 
        try :
            rate = driver.find_element_by_css_selector('#product_order_info > div.explan_product.product_info_section > ul > li:nth-child(7) > p.product_article_contents > a > span.prd-score__rating').text
            print(rate)
            item['rate'] = rate
            
        except:
            rate = None
            item['rate'] = rate
    
    # 구매 후기 개수 -> 평점과 구매 후기 개수를 곱해서 유의미한 feature을 만들어 낼 수 있을 것 같음 
    # 구매 후기 개수 또한 위와 같이 예외처리를 해주어야 한다. => 평점과 같은 위치에 있기 때문에
    
    try : 
        rate_num = driver.find_element_by_css_selector('#product_order_info > div.explan_product.product_info_section > ul > li:nth-child(6) > p.product_article_contents > a > span.prd-score__review-count').text
        rate_num = re.sub('[^0-9]', '', rate_num)
        print(rate_num)
        item['rate_num'] = rate_num
    
    except : 
        try :
            rate_num = driver.find_element_by_css_selector('#product_order_info > div.explan_product.product_info_section > ul > li:nth-child(7) > p.product_article_contents > a > span.prd-score__review-count').text
            rate_num = re.sub('[^0-9]', '', rate_num)
            print(rate_num)
            item['rate_num'] = rate_num
        except :
            rate_num = None
            item['rate_num'] = rate_num
            
    
    # 구매 현황 (purchase status)
    # ~18세 / 19 ~ 23세 / 24 ~ 28세 / 29 ~ 33세 / 34 ~ 39세 / 40세 ~ 
    
    try : 
        purchase_status = driver.find_element_by_css_selector('#page_product_detail > div.right_area.page_detail_product > div.section_graph_detail > div > div > div.graph_bar_wrap > div > ul').text
        purchase_status = purchase_status.split('\n') 

        cleaned_purchase_status = [elem for elem in purchase_status if '%' in elem]
        print(cleaned_purchase_status)
        item['purchase_status'] = cleaned_purchase_status
    
    except : 
        try : 
            purchase_status = driver.find_element_by_css_selector('#page_product_detail > div.right_area.page_detail_product > font > font > div.section_graph_detail > div > div > div.graph_bar_wrap > div > ul').text
            purchase_status = purchase_status.split('\n') 

            cleaned_purchase_status = [elem for elem in purchase_status if '%' in elem]
            print(cleaned_purchase_status)
            item['purchase_status'] = cleaned_purchase_status
            
        except :
            cleaned_purchase_status = None
            print(cleaned_purchase_status)
            item['purchase_status'] = cleaned_purchase_status
            
            
    
    # 남성 구매 비율 (파이 차트)
    
    try : 
        purchase_men = driver.find_element_by_css_selector('#graph_doughnut_label > ul > li:nth-child(1) > dl > dd').text
        print(purchase_men)
        item['purchase_men'] = purchase_men
    except :
        purchase_men = None
        print(purchase_men)
        item['purchase_men'] = purchase_men
    
    # 여성 구매 비율 (파이 차트)

    try :
        purchase_women = driver.find_element_by_css_selector('#graph_doughnut_label > ul > li:nth-child(2) > dl > dd').text
        print(purchase_women)
        item['purchase_women'] = purchase_women
    
    except :
        purchase_women = None
        print(purchase_women)
        item['purchase_women'] = purchase_women
    
    
    # 스크래핑한 딕셔너리 객체를 리스트에 추가해준다. 
    item_list.append(item)

    
    # 랜덤하게 대기를 해준다 (1~5초 사이)
    driver.implicitly_wait(random.randint(1, 5))
    
    # 구분선
    print('-'*20)
    
    
    


https://image.msscdn.net/images/goods_img/20210511/1944612/1944612_5_500.jpg?t=20220518142840
cut-heavy PIGMENT tshirts(CHARCOAL)
SOVERMENT
22summer-PT-03
2022 S/S / 남
53200
['피그먼트', '반팔', '반팔티', '티셔츠', '오버핏', '무지']
7797
4.9
2603
['5%', '26%', '34%', '21%', '8%', '6%']
89%
11%
--------------------
https://image.msscdn.net/images/goods_img/20220329/2453552/2453552_1_500.jpg?t=20220331173345
TSHIRT FLOWERMARDI_BLACK CREAM
MARDI MERCREDI
430767
2022 S/S / 여
42000
['그래픽']
20399
4.9
1755
['29%', '26%', '20%', '9%', '5%', '11%']
14%
86%
--------------------
https://image.msscdn.net/images/goods_img/20210412/1893766/1893766_2_500.jpg?t=20210427154411
백로고 3팩 반팔 티셔츠 BACKLOGO 3PACK TS
OUTDOOR PRODUCTS
WO136QCSSZ51
2022 S/S / 남 여
49900
None
24245
4.8
3976
['26%', '19%', '22%', '13%', '8%', '12%']
47%
53%
--------------------
https://image.msscdn.net/images/goods_img/20180403/747950/747950_1_500.jpg?t=20220321112716
[Mmlg] 19MG HF-T (WHITE)
MMLG
MMLGT123
남 여
42000
['로고', '반팔티', '반팔티셔츠', '팔칠엠엠', '하

https://image.msscdn.net/images/goods_img/20200423/1417716/1417716_2_500.jpg?t=20200630164139
우먼즈 크루 넥 크롭 반팔 티셔츠 [블랙]
MUSINSA STANDARD
MWJTS208-BK
여
11900
['기본티', '라운드넥', '무신사스탠다드우먼즈', '여름티', 'MUT', '크롭티', '크롭', '정호연PICK']
9477
4.8
11158
['16%', '29%', '28%', '13%', '6%', '8%']
5%
95%
--------------------
https://image.msscdn.net/images/goods_img/20220222/2376233/2376233_2_500.jpg?t=20220224121351
TAG KP TEE - WHITE
BROWNBREATH
BHMMST001WT
2022 S/S / 남 여
36000
['로고티', '반팔티셔츠', '로고티셔츠', '남자반팔티', '여자반팔티', '기본티셔츠', '그래픽티셔츠', '그래픽', '신스컬렉션']
12630
4.8
8624
['34%', '27%', '15%', '8%', '5%', '11%']
49%
51%
--------------------
https://image.msscdn.net/images/goods_img/20220310/2409398/2409398_1_500.jpg?t=20220312014622
T-Logo Tee White
THISISNEVERTHAT
TN220TTSST01WH
2022 S/S / 남 여
45000
['로고티셔츠', '티셔츠추천', '남자티셔츠', '티셔츠', '여름반팔티', '셀럽픽', '상의YOUAREIN']
17717
4.8
10565
['46%', '25%', '12%', '5%', '2%', '10%']
45%
55%
--------------------


In [145]:
len(item_list)
# json 객체의 길이는 92 -> 사실 90개인데 오류 났지만 담긴 것들도 있을 것이다. 

92

### 데이터 프레임 저장

In [146]:
#
# 이제 리스트 딕셔너리(json) 객체를 데이터 프레임으로 변환해보자
df = pd.DataFrame(item_list)
df

,src,brand_name,product_id,season_gender,price,hashtag,like,rate,rate_num,purchase_status,purchase_men,purchase_women
0,https://image.msscdn.net/images/goods_img/2021...,COVERNAT,CO0000STE1BK,2022 S/S / 남 여,49000,"[반팔티셔츠, 티셔츠, 반팔티, 오버핏반팔, 에센셜라인, 쿨코튼, 썸머컬렉션, 로고...",126121,4.8,32391,"[37%, 21%, 19%, 11%, 4%, 8%]",64%,36%
1,https://image.msscdn.net/images/goods_img/2021...,FP142,2125031,2022 S/S / 남,53800,"[베이직티셔츠, 오버핏, 그래픽티셔츠, 로고티셔츠, 반팔티, 오버핏티셔츠]",31440,4.8,2194,"[32%, 25%, 20%, 10%, 5%, 8%]",73%,27%
2,https://image.msscdn.net/images/goods_img/2019...,MUSINSA STANDARD,MITS0005-WH,남,11900,"[기본티, 라운드넥, 무지티, 반팔티, MUT, 홈웨어컬렉션, 유아인착장상품, 베이식]",61616,4.8,159122,"[27%, 26%, 22%, 12%, 6%, 7%]",80%,20%
3,https://image.msscdn.net/images/goods_img/2021...,YALE,YC2SS0041WH,2022 S/S / 남 여,39000,"[그래픽티셔츠, 오버핏반팔, 반팔티, 로고, 아이비리그, 그래픽, 최현욱착용]",63837,4.9,9658,"[43%, 25%, 14%, 6%, 2%, 10%]",58%,42%
4,https://image.msscdn.net/images/goods_img/2022...,YALE,YC1SS1021BK,2022 S/S / 남 여,78000,"[반팔티, 로고, 여름반팔티, 남자반팔티, 반팔티셔츠, 원마일웨어, 패키지셋업]",31729,4.9,6245,"[31%, 26%, 23%, 10%, 3%, 7%]",64%,36%
...,...,...,...,...,...,...,...,...,...,...,...,...
87,https://image.msscdn.net/images/goods_img/2018...,MMLG,MMLGT123,남 여,42000,"[로고, 반팔티, 반팔티셔츠, 로고티셔츠, 특별기획전, EVERYDAYWEAR, 그래픽]",30670,4.8,15665,"[32%, 29%, 20%, 8%, 3%, 8%]",46%,54%
88,https://image.msscdn.net/images/goods_img/2019...,UNIIS DESIGN,UNSD,ALL S/S / 남 여,39000,"[서핑, 티셔츠, 프린트, 썸머유니스, 그래픽, 프린팅, 써핑티]",27785,4.8,8459,"[24%, 27%, 25%, 14%, 5%, 5%]",71%,29%
89,https://image.msscdn.net/images/goods_img/2020...,MUSINSA STANDARD,MWJTS208-BK,여,11900,"[기본티, 라운드넥, 무신사스탠다드우먼즈, 여름티, MUT, 크롭티, 크롭, 정호연...",9477,4.8,11158,"[16%, 29%, 28%, 13%, 6%, 8%]",5%,95%
90,https://image.msscdn.net/images/goods_img/2022...,BROWNBREATH,BHMMST001WT,2022 S/S / 남 여,36000,"[로고티, 반팔티셔츠, 로고티셔츠, 남자반팔티, 여자반팔티, 기본티셔츠, 그래픽티셔...",12630,4.8,8624,"[34%, 27%, 15%, 8%, 5%, 11%]",49%,51%
